# Propensity to Buy Scoring Feed With Biased Data

This notebook can be scheduled to provide a continuous flow of data to your German Credit Risk model. Paste your Watson Machine Learning credentials below, and verify that the value of the **DEPLOYMENT_NAME** variable matches the name of your deployed German Credit Risk model. You can also set the maximum and minimum number of records that will be scored per run.

In [10]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client)


In [11]:
WML_CREDENTIALS = WML_CREDENTIALS ={
  "apikey": "wcPASQdXvtEPgA79lY4sYpCfiWAOoSNudy0hh24PMNbb",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/97deeb0b7e78431438a00a04f20580b7:e7e6948c-829b-4d59-ae09-3f991dfe52cf::",
  "iam_apikey_name": "auto-generated-apikey-f4fafb37-2a2d-477c-a9c6-0ff5400609f6",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/97deeb0b7e78431438a00a04f20580b7::serviceid:ServiceId-8e03beb2-f9d6-4351-b0e5-2add2a3437df",
  "instance_id": "e7e6948c-829b-4d59-ae09-3f991dfe52cf",
  "password": "d3538d26-7a6c-4e2f-b114-6a02041ce00b",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "f4fafb37-2a2d-477c-a9c6-0ff5400609f6"
}

In [12]:
DEPLOYMENT_NAME = "Propensity to Buy"
MIN_RECORDS = 1000
MAX_RECORDS = 1000

In [13]:
import json, random
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [14]:
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [15]:
!rm df_payload_biased.json
!wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_biased.json

--2019-03-11 16:04:45--  https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_biased.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249927 (244K) [text/plain]
Saving to: ‘df_payload_biased.json’

100%[======================================>] 249,927     --.-K/s   in 0.01s   

2019-03-11 16:04:45 (18.4 MB/s) - ‘df_payload_biased.json’ saved [249927/249927]



In [16]:
wml_deployments = wml_client.deployments.get_details()
scoring_url = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        scoring_url = deployment['entity']['scoring_url']
        break
    
print("Scoring URL: {}".format(scoring_url))

Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e7e6948c-829b-4d59-ae09-3f991dfe52cf/deployments/e5010977-c901-4e07-a474-11b77b7c7d8c/online


In [17]:
try:
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file found')
    
except:
    !wget https://raw.githubusercontent.com/shadgriffin/A-DGSCLAB/master/df_payload_biased.json
    with open('df_payload_biased.json', 'r') as scoring_file:
        scoring_data = json.load(scoring_file)
    print('file downloaded')

file found


In [18]:
fields = scoring_data['fields']
values = []
for _ in range(0, random.randint(MIN_RECORDS, MAX_RECORDS)):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)
print(scoring_response)

{'fields': ['INCOME', 'AGE_IN_YEARS', 'HAS_CHILDREN', 'RENTER', 'OWNER', 'GOOD_CREDIT', 'COLLEGE_DEGREE', 'AVG_DIST_FROM_HOME', 'MILLENIAL', 'MID_LEVEL_AUTO', 'DISTANCE_FROM_HOME', 'TRAVELER', 'TIME_OF_DAY', 'BUY_LABEL', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [[80372.1774753994, 40.893539, 0.0, 0, 1, 1, 0, 17.82, 0, 0, 13, 0, 14, 'BUY', 0.0, [13, [0, 1, 4, 5, 7, 10, 12], [80372.1774753994, 40.893539, 1.0, 1.0, 17.82, 13.0, 14.0]], [8.96676743561104, 11.03323256438896], [0.44833837178055197, 0.551661628219448], 1.0, 'NOT_BUY'], [80372.1840753994, 55.4516022178, 0.0, 0, 0, 0, 0, 13.92, 0, 0, 8, 0, 10, 'BUY', 0.0, [13, [0, 1, 7, 10, 12], [80372.1840753994, 55.4516022178, 13.92, 8.0, 10.0]], [5.83495973345284, 14.16504026654716], [0.291747986672642, 0.708252013327358], 1.0, 'NOT_BUY'], [462933.0, 42.223797, 1.0, 0, 1, 0, 0, 39.48, 0, 1, 2, 0, 21, 'BUY', 0.0, [462933.0, 42.223797, 1.0, 0.0, 1.0, 0.0, 0.0, 39.48, 0.0, 1.0, 2.0, 0.0, 21